# Setup

In [3]:
import sqlite3, random
import pandas as pd
df = pd.read_csv('rct_df.csv', index_col=0)
df = df[['id','study.name','specialty','outcome_name','comparison_name','microsoft/Phi-3.5-mini-instruct_probs']]

#-- add label col
df['label'] = pd.Series(dtype='str')

#-- add q
question = f"In a medical study on '{df.comparison_name}', the outcome is '{df.outcome_name}'. Is this considered a positive or negative health outcome? Please only respond with 'positive', 'negative' or 'neutral'."

In [4]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect('cochrane_labeling.db')

#-- define cursor
cur = conn.cursor()

# Write the DataFrame to SQL (replace if it already exists)
df.to_sql('rct_df', conn, if_exists='replace', index=False)

# #-- check if written
# res = cur.execute("SELECT name FROM sqlite_master")
# res.fetchone()

#-- check if column is created
cur.execute("ALTER TABLE 'rct_df' ADD COLUMN status TEXT DEFAULT 'available'")
res = cur.execute("SELECT status FROM rct_df")
res.fetchall()

#Close the connection
conn.close()


In [ ]:

import random

def sample_questions(conn, user_id, num_samples=4):
    cursor = conn.cursor()
    
    # Select rows with score <= 50 that are not being labeled
    cursor.execute("SELECT id FROM questions WHERE score <= 50 AND status = 'available'")
    easy_questions = cursor.fetchall()
    
    # Select rows with score > 50 that are not being labeled
    cursor.execute("SELECT id FROM questions WHERE score > 50 AND status = 'available'")
    hard_questions = cursor.fetchall()

    # Randomly sample 3 easy and 1 hard question
    sampled_questions = random.sample(easy_questions, min(3, len(easy_questions))) + \
                        random.sample(hard_questions, min(1, len(hard_questions)))

    # Mark them as 'in-progress' for the user
    for question in sampled_questions:
        cursor.execute("UPDATE questions SET status = 'in-progress', user_id = ? WHERE id = ?", (user_id, question[0]))
    
    # Commit the changes
    conn.commit()
    return sampled_questions


In [ ]:
#-- create a sampler